In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from transformers import pipeline

from nltk import sent_tokenize
import nltk
from time import time
import matplotlib.pyplot as plt
import torch

C:\Users\gabri\anaconda3\envs\web-scrap\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
C:\Users\gabri\anaconda3\envs\web-scrap\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# Load Dataset

In [3]:
subtitles_paths = sorted(glob("../data/subtitles/*.ass"))

In [4]:
subtitles_paths[:5]

['../data/subtitles\\Naruto Season 1 - 01.ass',
 '../data/subtitles\\Naruto Season 1 - 02.ass',
 '../data/subtitles\\Naruto Season 1 - 03.ass',
 '../data/subtitles\\Naruto Season 1 - 04.ass',
 '../data/subtitles\\Naruto Season 1 - 05.ass']

In [5]:
scripts = []
episode_num = []
for path in subtitles_paths:
    
    with open(path, "r", encoding="utf-8") as f:
        lines = f.readlines()
        lines = lines[27:]
        
        rows = [",".join(line.split(',')[9:]) for line in lines]
    
    rows = [line.replace("\\N", "") for line in rows]
    script = " ".join(rows)
    
    episode = int(path.split('-')[1].split('.')[0].strip())
    
    scripts.append(script)
    episode_num.append(episode)

In [6]:
df = pd.DataFrame.from_dict({'episode':episode_num,'script':scripts})

In [7]:
df.head(5)

,episode,script
0,1,"A long time ago, a powerful demon foxappeared ..."
1,2,"C'mon!\n Running like a fugitive,\n Being chas..."
2,3,"C'mon!\n Running like a fugitive,\n Being chas..."
3,4,"C'mon!\n Running like a fugitive,\n Being chas..."
4,5,"C'mon!\n Running like a fugitive,\n Being chas..."


# Load Model

In [8]:
device = 0 if torch.cuda.is_available() else 'cpu'
print(device)

0


In [9]:
from transformers import pipeline
theme_classifier = pipeline("zero-shot-classification",
                      model= 'facebook/bart-large-mnli',
                      device=device)

C:\Users\gabri\anaconda3\envs\web-scrap\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
theme_classifier("I have a problem with my iphone that needs to be resolved asap!",
    candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"],
    multi_label=False
)

{'sequence': 'I have a problem with my iphone that needs to be resolved asap!',
 'labels': ['urgent', 'phone', 'computer', 'not urgent', 'tablet'],
 'scores': [0.5227583646774292,
  0.45813968777656555,
  0.014264845289289951,
  0.0026850055437535048,
  0.0021520759910345078]}

# Inference

In [11]:
def get_themes(script):
    script_sentences = sent_tokenize(script)

    # Batch sentences 
    script_batches = []
    sentence_batch_size=20
    for index in range(0,len(script_sentences),sentence_batch_size):
        sent = " ".join(script_sentences[index:index+sentence_batch_size])
        script_batches.append(sent)
    
    # Run model
    candidate_labels = ['friendship','hope', 'sacrifice', 'battle','self development','betrayal','love','dialogue'] 
    emotion_output = theme_classifier(script_batches, 
                       candidate_labels,multi_label=True)

    # Wrangle Output    
    emotions={}
    for output in  emotion_output:
        for label, score in zip(output['labels'],output['scores']):
            if label not in emotions:
                emotions[label]=[]
            emotions[label].append(score)
    
    emotions =  {key:np.mean(np.array(value)) for key,value in emotions.items()}
   
    return emotions

In [12]:
output_emotions = df['script'].apply(get_themes)

C:\Users\gabri\anaconda3\envs\web-scrap\Lib\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [13]:
emotion_df = pd.DataFrame(output_emotions.tolist())

In [14]:
df[emotion_df.columns] = emotion_df

In [15]:
theme_output = df.drop(['episode','script','dialogue'],axis=1).sum().reset_index()
theme_output.columns = ['theme','score']
theme_output

,theme,score
0,betrayal,118.840843
1,battle,153.248394
2,self development,145.221716
3,sacrifice,156.423085
4,hope,84.386500
5,friendship,71.730532
6,love,44.767524


In [ ]:
sns.barplot(data=theme_output, x="theme", y="score")
plt.xticks(rotation=45)
plt.show()